In [ ]:
print("Hello Terra")

### Log in as assigned UAMI

### Get the JWT

In [ ]:
%%bash --out cli
az account get-access-token

In [ ]:
import json
accessToken = json.loads(cli)['accessToken']
accessToken

### Call Sam userInfo

In [ ]:
import requests
endpoint = 'https://sam.fiab-rtitle-smooth-wren.bee.envs-terra.bio/register/user/v2/self/info'
headers = {
    "Authorization": f"Bearer {accessToken}",
    "Accept": "application/json"
}
res = requests.get(endpoint, headers=headers)
print(res.json())

### Call WSM getSasToken

In [ ]:
WSM_BASE_URL = 'https://workspace.fiab-rtitle-smooth-wren.bee.envs-terra.bio/api/workspaces/v1'
WSM_BASE_HEADERS = {"Authorization": f"Bearer {accessToken}", "Accept": "application/json"}

# TODO: it's hard to navigate WSM APIs to get workspace id, storage account/container name/id, signed url
# Can we pass some information to the notebook as EVs?

# Get WSM workspace id
wsm_workspace_list = requests.get(f'{WSM_BASE_URL}', headers=WSM_BASE_HEADERS).json()
wsm_workspace_id = next(filter(lambda x: x['displayName'] == 'Rob Azure Demo 5', wsm_workspace_list['workspaces']))['id']
print(f"wsm workspace id: {wsm_workspace_id}")

# List storage accounts in the workspace
wsm_storage_account_list = requests.get(f'{WSM_BASE_URL}/{wsm_workspace_id}/resources', headers=WSM_BASE_HEADERS).json()
wsm_storage_account_name = next(filter(lambda x: x['metadata']['resourceType'] == 'AZURE_STORAGE_ACCOUNT', wsm_storage_account_list['resources']))['metadata']['name']
print(f"wsm storage account name: {wsm_storage_account_name}")

# List storage containers in the workspace
wsm_storage_container_list = requests.get(f'{WSM_BASE_URL}/{wsm_workspace_id}/resources', headers=WSM_BASE_HEADERS).json()
wsm_storage_container_id = next(filter(lambda x: x['metadata']['resourceType'] == 'AZURE_STORAGE_CONTAINER', wsm_storage_container_list['resources']))['metadata']['resourceId']
print(f"wsm storage container id: {wsm_storage_container_id}")

# Get SAS token from WSM
wsm_sas_token_response = requests.post(f"{WSM_BASE_URL}/{wsm_workspace_id}/resources/controlled/azure/storageContainer/{wsm_storage_container_id}/getSasToken", headers=WSM_BASE_HEADERS).json()
sas_token = wsm_sas_token_response.get('token')

# Build signed url
signed_url_base = f"https://sa5f2f225a50952a945dcf.blob.core.windows.net/{wsm_storage_account_name.replace('sa', 'sc')}"
signed_url = f"{signed_url_base}?{sas_token}"
print(f"signed url base is {signed_url_base}")
print(f"sas token is {sas_token}")

### Read/write data from SAS token

In [ ]:
# copy this notebook file to the storage container
blob_url = f"{signed_url_base}/Demo.ipynb?{sas_token}"
! azcopy copy 'Demo.ipynb' '{blob_url}'

In [ ]:
# copy it back
import time
new_filename = 'Demo.ipynb.' + time.strftime("%Y%m%d-%H%M%S")
! azcopy copy '{blob_url}' '{new_filename}'

In [ ]:
!ls -la Demo.ipynb*